## K Nearest Neighbors

In [17]:
import os
import settings
import pandas as pd
import numpy as np
from scipy.spatial import distance


In [18]:
def read_data():
    df = pd.read_excel(os.path.join(settings.PROCESSED_DIR, "all_with_liwc_segmented.xls"), encoding="ISO-8859-1")
    return df

In [19]:
df = read_data()

In [27]:
df_normalized = (numerical - numerical.mean()) / numerical.std()

In [29]:
df_normalized.describe()

,index,comments,duration,film_date,languages,published_date,views,music,conversation,persuasive,...,affect_change_h,posemo_change_q,negemo_change_q,affect_change_q,published_year,Harm,Fairness,Purity,Ingroup,Authority
count,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,0.0,0.0,2.406000e+03,...,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03,2.406000e+03
mean,-2.803705e-16,-1.407390e-17,-1.974960e-17,5.507740e-16,1.675948e-16,-1.040915e-15,-1.938045e-17,NaN,NaN,1.295491e-16,...,-6.720411e-16,3.890856e-16,5.969871e-17,-1.657029e-16,2.731850e-14,-2.742553e-15,-1.682869e-16,-1.355432e-15,-4.223093e-16,4.155723e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,NaN,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.743743e+00,-6.565752e-01,-2.151630e+00,-7.579897e+00,-3.078753e+00,-2.035764e+00,-6.201359e-01,NaN,NaN,-4.779383e-01,...,-8.808653e+00,-6.672823e+00,-4.429294e+00,-4.135881e+00,-2.032441e+00,-7.647945e-01,-4.923961e-01,-5.214786e-01,-1.021057e+00,-8.847681e-01
25%,-8.666380e-01,-4.549047e-01,-7.268197e-01,-5.495643e-01,-5.525234e-01,-8.016317e-01,-3.702101e-01,NaN,NaN,-3.910931e-01,...,-5.727933e-01,-5.775448e-01,-4.768718e-01,-6.284909e-01,-7.377485e-01,-6.347364e-01,-4.923961e-01,-5.214786e-01,-6.831529e-01,-6.139689e-01
50%,2.955569e-02,-2.619268e-01,9.749004e-02,9.096889e-02,2.161986e-02,-6.424434e-03,-2.325614e-01,NaN,NaN,-2.652990e-01,...,-4.335724e-02,-5.302528e-02,6.035060e-02,-4.718255e-02,-9.040244e-02,-3.095909e-01,-4.923961e-01,-2.887430e-01,-2.776682e-01,-2.271130e-01
75%,8.638748e-01,1.014278e-01,7.011432e-01,8.261768e-01,5.957631e-01,8.451070e-01,-8.946110e-03,NaN,NaN,1.207945e-02,...,5.418980e-01,5.317838e-01,4.747793e-01,5.426745e-01,8.806166e-01,2.675423e-01,8.761055e-02,1.185443e-01,3.643491e-01,2.757996e-01
max,1.714650e+00,2.160368e+01,5.864506e+00,1.546592e+00,5.074080e+00,1.671422e+00,1.792549e+01,NaN,NaN,2.205761e+01,...,4.452621e+00,1.457323e+01,1.925454e+01,5.658432e+00,1.527963e+00,9.737404e+00,1.425634e+01,1.443178e+01,5.737021e+00,1.052748e+01


In [68]:
def find_similar_speaker(name):
    numerical = df.select_dtypes(include=['int64','float64'])
    df_normalized = (numerical - numerical.mean()) / numerical.std()
    df_normalized.drop(['conversation','music','index','languages','comments', 'duration', 'views','persuasive', 'unconvincing', 'inspiring', 'film_date', 'published_date','published_year','AllPunc','Period','Comma','Colon','SemiC','QMark','Exclam','Dash','Quote','Apostro','Parenth','OtherP', 'affect_1h','posemo_1h','negemo_1h', 'anx_1h', 'anger_1h', 'sad_1h', 'affect_2h', 'posemo_2h', 'negemo_2h', 'anx_2h', 'anger_2h', 'sad_2h', 'affect_1q', 'posemo_1q', 'negemo_1q', 'anx_1q', 'anger_1q', 'sad_1q', 'affect_2q', 'posemo_2q', 'negemo_2q', 'anx_2q', 'anger_2q', 'sad_2q', 'affect_3q', 'posemo_3q', 'negemo_3q', 'anx_3q', 'anger_3q', 'sad_3q', 'affect_4q', 'posemo_4q','negemo_4q', 'anx_4q', 'anger_4q','sad_4q','posemo_change_h', 'negemo_change_h','affect_change_h', 'posemo_change_q', 'negemo_change_q', 'affect_change_q'],axis=1,inplace=True)
    print(df_normalized.describe())
    speaker_normalized = df_normalized[df["main_speaker"] == name]
    speaker_normalized = speaker_normalized.iloc[0]
    euclidean_distances = df_normalized.apply(lambda row: distance.euclidean(row, speaker_normalized), axis=1)
    print(euclidean_distances.nsmallest(5))
    second_smallest_value = euclidean_distances.nsmallest(2).iloc[1]
    rec_idx = euclidean_distances[euclidean_distances == second_smallest_value].index
    return df[['main_speaker', 'description', 'url' ]].iloc[rec_idx]

In [69]:
find_similar_speaker('Al Gore')

           applause      laughter  norm_persuasive  norm_inspiring  \
count  2.406000e+03  2.406000e+03     2.406000e+03    2.406000e+03   
mean   6.158369e-16 -3.133634e-16     1.249855e-15   -1.870906e-16   
std    1.000000e+00  1.000000e+00     1.000000e+00    1.000000e+00   
min   -1.088511e+00 -6.717769e-01    -8.195969e-01   -1.019686e+00   
25%   -5.658695e-01 -6.717769e-01    -5.951708e-01   -6.232306e-01   
50%   -5.658695e-01 -3.413476e-01    -2.760374e-01   -2.555344e-01   
75%    4.794142e-01  1.542965e-01     2.573260e-01    3.648244e-01   
max    8.319042e+00  1.155411e+01     1.566080e+01    2.349784e+01   

       norm_unconvincing            WC      Analytic         Clout  \
count       2.406000e+03  2.406000e+03  2.406000e+03  2.406000e+03   
mean       -9.649619e-16  1.235735e-16 -1.264724e-15  7.686933e-15   
std         1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
min        -5.342445e-01 -2.289878e+00 -2.948052e+00 -5.225593e+00   
25%        -4.19164

,main_speaker,description,url
1822,Mac Barnett,Childhood is surreal. Why shouldn't children's...,https://www.ted.com/talks/mac_barnett_why_a_go...


In [70]:
def find_similar_talk_moral(title):
    numerical = df[['HarmVirtue', 'HarmVice', 'AuthorityVirtue', 'AuthorityVice', 'IngroupVirtue', 'IngroupVice', 'FairnessVirtue', 'FairnessVice', 'PurityVirtue', 'PurityVice', 'MoralityGeneral']]
    df_normalized = (numerical - numerical.mean()) / numerical.std()
    speaker_normalized = df_normalized[df["title"] == title]
    speaker_normalized = speaker_normalized.iloc[0]
    print(speaker_normalized)
    euclidean_distances = df_normalized.apply(lambda row: distance.euclidean(row, speaker_normalized), axis=1)
    print(euclidean_distances.nsmallest(5))
    second_smallest_value = euclidean_distances.nsmallest(2).iloc[1]
    rec_idx = euclidean_distances[euclidean_distances == second_smallest_value].index
    #print(euclidean_distances.sort_values())
    return df[['main_speaker', 'description', 'url']].iloc[rec_idx]

In [71]:
find_similar_talk_moral('Our loss of wisdom')

HarmVirtue         0.658628
HarmVice          -0.475024
AuthorityVirtue    0.892933
AuthorityVice      0.374477
IngroupVirtue      0.055593
IngroupVice       -0.436870
FairnessVirtue     0.303447
FairnessVice      -0.243597
PurityVirtue      -0.070717
PurityVice        -0.161798
MoralityGeneral    3.856452
Name: 384, dtype: float64
384     0.000000
1976    1.498020
2451    1.607556
1049    1.693742
1596    1.743696
dtype: float64


,main_speaker,description,url
2163,Arthur Brooks,Conservatives and liberals both believe that t...,https://www.ted.com/talks/arthur_brooks_a_cons...


In [47]:
my_col = list(df_normalized.columns)

In [48]:
my_col

['index',
 'comments',
 'duration',
 'film_date',
 'languages',
 'published_date',
 'views',
 'music',
 'conversation',
 'persuasive',
 'inspiring',
 'unconvincing',
 'applause',
 'laughter',
 'norm_persuasive',
 'norm_inspiring',
 'norm_unconvincing',
 'WC',
 'Analytic',
 'Clout',
 'Authentic',
 'Tone',
 'WPS',
 'Sixltr',
 'Dic',
 'function',
 'pronoun',
 'ppron',
 'i',
 'we',
 'you',
 'shehe',
 'they',
 'ipron',
 'article',
 'prep',
 'auxverb',
 'adverb',
 'conj',
 'negate',
 'verb',
 'adj',
 'compare',
 'interrog',
 'number',
 'quant',
 'affect',
 'posemo',
 'negemo',
 'anx',
 'anger',
 'sad',
 'social',
 'family',
 'friend',
 'female',
 'male',
 'cogproc',
 'insight',
 'cause',
 'discrep',
 'tentat',
 'certain',
 'differ',
 'percept',
 'see',
 'hear',
 'feel',
 'bio',
 'body',
 'health',
 'sexual',
 'ingest',
 'drives',
 'affiliation',
 'achieve',
 'power',
 'reward',
 'risk',
 'focuspast',
 'focuspresent',
 'focusfuture',
 'relativ',
 'motion',
 'space',
 'time',
 'work',
 'leisur